In [213]:
import pypdf
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   

from pathlib import Path
from functools import partial
from langchain_openai import  AzureChatOpenAI
from langchain.prompts import PromptTemplate
from utils_ccd import get_connection

conn = get_connection()

In [303]:
DIR_INFORMACOES = '/media/informacoes_pdf/'

def get_file_path(row):
    return Path(DIR_INFORMACOES) / row['setor'].strip() / row['arquivo']

def extract_text_from_pdf(file_path):
    try:
        with open(file_path, 'rb') as f:
            reader = pypdf.PdfReader(f)
            text = ''
            for page in reader.pages:
                text += page.extract_text() or ''
            return text
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ''
    
def get_next_informacao(processo, df_last, df_informacoes):
    try:
        evento = df_last[df_last['processo'] == \
                                         processo]['evento'].iloc[0]
        return df_informacoes[(df_informacoes.evento > \
                                      evento) & \
                                        (df_informacoes['processo'] == \
                                         processo)].sort_values(by='evento').iloc[0]
    except IndexError:
        return None

def get_text_next_informacao(processo, df_last, df_informacoes):
    info = get_next_informacao(processo, df_last, df_informacoes)
    if info is not None:
        return extract_text_from_pdf(info['caminho_arquivo'])
    return ''


llm = AzureChatOpenAI(model_name="gpt-4o")

def get_descricao_objeto(texto):
    try:
        prompt = PromptTemplate.from_template("""
        Você é um agente que analisa e resume informações de processos do Tribunal de Contas do Estado do Rio Grande do Norte.

        O texto da informação é o seguinte:
        "{input}"
        Defina a Descrição resumida do objeto/assunto principal. 
        Seja sucinto e claro, não é necessário mencionar valores ou números de processos ou leis.
        Deve ser uma frase com no máximo 7 palavras, sem muitos detalhes.
                                              
        Por exemplo: 
            Cancelamento por prescrição. 
            Atualização do CADINQ. 
            Implantação de débito de desconto em folha.
            Instauração de Processo de Execução.  

        Sua resposta:
        """)

        chain = prompt | llm
        msg_obj = chain.invoke(texto)
        return msg_obj.content
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None
    
def get_descricao_encaminhamento(texto, proximo_texto):
    if not proximo_texto:
        return f'Nenhum encaminhamento até o momento {datetime.datetime.now().strftime("%d/%m/%Y")}'
    try:
        prompt = PromptTemplate.from_template("""
        Você é um agente que analisa encaminhamentos. A informação é sobre um 
        processo do Tribunal de Contas do Estado do Rio Grande do Norte.
        
        O texto da primeira informação é o seguinte:
        "{texto}"

        Depois veio a seguinte informação de encaminhamento:                                      
        "{proximo_texto}"
                                            
        Seja sucinto e claro, não é necessário mencionar valores ou números de processos ou leis.
        Deve ser uma frase ou duas, sem muitos detalhes.
                                              
        Por exemplo: 
            Encaminhamento do Ministério Público à Procuradoria Geral do Estado.
             
            
                                              
        Sua resposta:
        """)

        chain = prompt | llm
        msg_obj = chain.invoke({"texto":texto, "proximo_texto":proximo_texto})
        return msg_obj.content
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None

def generate_excel(setor, ano, mes, filename):
    sql_informacoes_processos = f'''
    SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
    ppe.SequencialProcessoEvento as evento,
    CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
    inf.*
    FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON inf.idinformacao = ppe.idinformacao
    WHERE setor = '{setor}'
    and year(inf.data_resumo) = {ano}
    and month(inf.data_resumo) = {mes}
    '''
    informacoes_processos = pd.read_sql(sql_informacoes_processos, conn)
    informacoes_processos['caminho_arquivo'] = informacoes_processos.apply(get_file_path, axis=1)
    
    informacoes_processos = informacoes_processos[~informacoes_processos['resumo'].str.contains('Pagamento da parcela')]
    informacoes_processos = informacoes_processos[~informacoes_processos['resumo'].str.contains('Capa')]
    informacoes_processos = informacoes_processos[~informacoes_processos['resumo'].str.contains('Evento do Processo Original')]

    informacoes_setor = informacoes_processos[informacoes_processos['setor'].str.strip() == setor]
    informacoes_setor_last = informacoes_setor.sort_values(by='evento', ascending=False).drop_duplicates(subset='processo', keep='first')

    informacoes_setor_last['resumo'].fillna('', inplace=True)
    
    informacoes_setor_last = informacoes_setor_last[~informacoes_setor_last['resumo'].str.contains('Pagamento da parcela')]

    informacoes_setor_last['texto'] = informacoes_setor_last['caminho_arquivo'].apply(extract_text_from_pdf)

    processos_busca = ', '.join(f"'{proc}'" for proc in informacoes_setor_last['processo'].unique())

    sql_all_informacoes_processos = f'''
    SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
    ppe.SequencialProcessoEvento as evento,
    CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
    inf.*
    FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON inf.idinformacao = ppe.idinformacao
    WHERE concat(inf.numero_processo, '/', inf.ano_processo) in ({processos_busca})
    '''
    all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)
    all_informacoes_processos['caminho_arquivo'] = all_informacoes_processos.apply(get_file_path, axis=1)

    get_text_next_informacao_partial = partial(get_text_next_informacao, \
                                               df_last=informacoes_setor_last, \
                                                df_informacoes=all_informacoes_processos)
    informacoes_setor_last['proximo_texto'] = informacoes_setor_last['processo'].apply(get_text_next_informacao_partial)

    informacoes_setor_last['descricao_objeto'] = informacoes_setor_last['texto'].apply(get_descricao_objeto)
    informacoes_setor_last['descricao_encaminhamento'] = informacoes_setor_last.apply(
        lambda row: get_descricao_encaminhamento(row['texto'], row['proximo_texto']), axis=1
    )
    
    sql_processos = f'''
       SELECT CONCAT(p.numero_processo, '/', p.ano_processo) AS processo,
            p.interessado AS jurisdicionado,
            rel.nome as relator,
            p.assunto,
            (SELECT TOP 1 il.recebido_em
	            FROM processo.dbo.Lotes AS l
	            INNER JOIN processo.dbo.Itens_Lote AS il 
	            	ON il.IdLote = l.IdLote
	            INNER JOIN processo.dbo.Processos AS p2 
	            	ON p2.numero_processo = il.numero_processo AND p2.ano_processo = il.ano_processo
	            WHERE p2.numero_processo = p.numero_processo AND p2.ano_processo = p.ano_processo
	            AND destino = '{setor}'
	            ORDER BY l.enviado_em DESC
            ) as data_recebimento,
            (SELECT TOP 1 l.enviado_em
	            FROM processo.dbo.Lotes AS l
	            INNER JOIN processo.dbo.Itens_Lote AS il 
	            	ON il.IdLote = l.IdLote
	            INNER JOIN processo.dbo.Processos AS p2 
	            	ON p2.numero_processo = il.numero_processo AND p2.ano_processo = il.ano_processo
	            WHERE p2.numero_processo = p.numero_processo AND p2.ano_processo = p.ano_processo
	            AND origem = '{setor}'
	            ORDER BY l.enviado_em DESC
            ) as data_saida
            
            
	FROM processo.dbo.Processos p
	LEFT JOIN processo.dbo.Relator AS rel ON p.codigo_relator = rel.codigo
	WHERE CONCAT(p.numero_processo, '/', p.ano_processo) in ({processos_busca})
        '''
    df_processos = pd.read_sql(sql_processos, conn)

    processos_informacoes_setor = informacoes_setor_last.merge(df_processos, on='processo', how='left')
    
    tab_setor = processos_informacoes_setor[['processo', 'jurisdicionado', 'relator', 
                            'data_recebimento', 'data_saida', 
                            'descricao_objeto', 'descricao_encaminhamento']]
    tab_setor.fillna('', inplace=True)
    tab_setor.to_excel(filename, index=False)
    

In [304]:
generate_excel('CCD', 2025, 4, 'saidas/sisdgf/processos_ccd.xlsx')

/tmp/ipykernel_59025/395666742.py:109: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  informacoes_processos = pd.read_sql(sql_informacoes_processos, conn)
/tmp/ipykernel_59025/395666742.py:136: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


Error processing Processo nº:002015/2025-TC
Assunto: EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PROCESSO Nº 004373/2016 - TC .
 
  
DESPACHO
 
              Considerando que foram tomadas todas as medidas que competem a esta Diretoria 
quanto à execução das dívidas do presente processo, especialmente no concerne ao 
PROTESTO CARTORIAL, o qual apesar de efetivo, não obteve êxito no recolhimento do valor 
devido. Informo ainda que, em pesquisa junto ao SIAI DP, Portal da Transparência do Governo 
Federal, Portal da Transparência da Câmara dos Deputados e Senado Federal,  não foi 
encontrado vínculo do responsável com a administração pública na presente data.
 
               Com estas informações, encaminhe-se ao  Ministério Público junto ao Tribunal de 
Contas para providências de sua alçada quanto à continuidade da execução das multas e /ou 
ressarcimentos.
    
Natal/RN, 28 de abril de 2025.
                                                                                              

/tmp/ipykernel_59025/395666742.py:180: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_processos = pd.read_sql(sql_processos, conn)
/tmp/ipykernel_59025/395666742.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tab_setor.fillna('', inplace=True)


In [170]:
generate_excel('CIP', 2025, 4, 'saidas/sisdgf/processos_cip.xlsx')

/tmp/ipykernel_59025/1677584847.py:141: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_processos = pd.read_sql(sql_processos, conn)
/tmp/ipykernel_59025/1677584847.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  informacoes_processos = pd.read_sql(sql_informacoes_processos, conn)
/tmp/ipykernel_59025/1677584847.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tab_setor.fillna('', inplace=True)


In [218]:
setor = 'CCD'
filename = 'saidas/sisdgf/processos_ccd.xlsx'
ano = 2025
mes = 4

In [264]:
sql_informacoes_processos = f'''
SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
ppe.SequencialProcessoEvento as evento,
CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
inf.*
FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
    ON inf.idinformacao = ppe.idinformacao
WHERE setor = '{setor}'
and year(inf.data_resumo) = {ano}
and month(inf.data_resumo) = {mes}
'''
informacoes_processos = pd.read_sql(sql_informacoes_processos, conn)
informacoes_processos['caminho_arquivo'] = informacoes_processos.apply(get_file_path, axis=1)
informacoes_processos['processo'] = informacoes_processos['numero_processo'] + '/' + informacoes_processos['ano_processo'].astype(str)
informacoes_processos = informacoes_processos[~informacoes_processos['resumo'].str.contains('Pagamento da parcela')]
informacoes_processos = informacoes_processos[~informacoes_processos['resumo'].str.contains('Capa')]
informacoes_processos = informacoes_processos[~informacoes_processos['resumo'].str.contains('Evento do Processo Original')]

informacoes_setor = informacoes_processos[informacoes_processos['setor'].str.strip() == setor]
informacoes_setor_last = informacoes_setor.sort_values(by='evento', ascending=False).drop_duplicates(subset='processo', keep='first')

informacoes_setor_last['resumo'].fillna('', inplace=True)


informacoes_setor_last['texto'] = informacoes_setor_last['caminho_arquivo'].apply(extract_text_from_pdf)

get_text_next_informacao_partial = partial(get_text_next_informacao, df_last=informacoes_setor_last, df_processos=informacoes_processos)
informacoes_setor_last['proximo_texto'] = informacoes_setor_last['processo'].apply(get_text_next_informacao_partial)

/tmp/ipykernel_59025/804071507.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  informacoes_processos = pd.read_sql(sql_informacoes_processos, conn)


In [277]:
processos_busca = ', '.join(f"'{proc}'" for proc in informacoes_setor_last['processo'].unique())

In [292]:
processos_busca

"'000414/2019', '002015/2025', '009673/2015', '000671/2016', '001820/2020', '007939/2018', '000931/2020', '009276/2018', '012015/2015', '200209/2021', '012695/2017', '200069/2021', '200044/2021', '000731/2025', '006411/2019', '008522/2018', '000804/2019', '003343/2017', '002954/2018', '200094/2022', '004832/2020', '003356/2024', '200105/2022', '200189/2021', '003911/2019', '100015/2021', '003020/2022', '007405/2019', '001380/2021', '003049/2022', '002157/2019', '000086/2022', '000088/2021', '003362/2024', '002077/2020', '700790/2012', '002056/2020', '002016/2025', '001296/2020', '001368/2021', '008464/2014', '008429/2014', '007426/2019', '000852/2025', '007432/2019', '002012/2025', '011947/2012', '002022/2024', '002071/2020', '000721/2025', '000856/2025', '200007/2023', '002011/2025', '002842/2014', '000854/2025', '000066/2025', '000144/2022', '007043/2019', '004974/2009', '002178/2019', '000861/2025', '000850/2025', '005299/2017', '002572/2024', '003392/2024', '002606/2024', '003001/1

In [293]:
sql_all_informacoes_processos = f'''
    SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
    ppe.SequencialProcessoEvento as evento,
    CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
    inf.*
    FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON inf.idinformacao = ppe.idinformacao
    WHERE concat(inf.numero_processo, '/', inf.ano_processo) in ({processos_busca})
    '''
all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)

/tmp/ipykernel_59025/3646662381.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


In [294]:
all_informacoes_processos['setor'].unique()

array(['DAE_SEI', 'PROC_CJ', 'DE        ', 'DAE_SEI   ', 'DAM_FGO', 'DAE',
       'DAE_EXE   ', 'GCADE', 'DAE       ', 'GCGIL', 'DAI', 'GCTAR',
       'DAE_EXP', 'GCREN', 'DE', 'DAE_EXE', 'GAANA', 'GAMAR', 'PROC_LRC',
       'DAM_FGE', 'GCPRO', 'SECEX', 'DAE_MANDA', 'GCCTH', 'DAM',
       'DAE_EXP   ', 'PROC      ', 'GCPOT', 'DDP', 'PROC_GCB  ',
       'PROC_EXE  ', 'PROC_PC', 'PROC_EXE', 'PROC_GCB', 'CONJU',
       'CONJU_NC', 'PROC_POM', 'PROC_GRC', 'DAE_SOB', 'GAANA     ',
       'secpc', 'secss', 'GCREN     ', 'GAANT     ', 'secsc',
       'PROC_EXSOB', 'PROC_PLS', 'GCTAR     ', 'SECPC', 'GAANT', 'DAP',
       'SECSS', 'GCCTH     ', 'GCADE     ', 'SECSC', 'DAP_BEN',
       'GCGIL     ', 'PROC_PC   ', 'GAMAR     ', 'DAE_MANDA ',
       'GCPRO     ', 'COEX', 'PROC_CTJ', 'PROC_ADJ', 'DIN', 'GCCED',
       'GCCED     ', 'PROC_PTM', 'GCCAP', 'GCGEO', 'GCAED     ', 'GCAED',
       'CCD       ', 'GCGEO     ', 'GCPOT     ', 'CCD', 'DE_EXP',
       'DE_MANDA', 'GCCORR', 'GCCORR    ', 'DE_MA

In [295]:
df_last = informacoes_setor_last
df_processos = all_informacoes_processos

In [296]:
processo = '700790/2012'

In [297]:
evento = df_last[df_last['processo'] == \
                                         processo]['evento'].iloc[0]


In [298]:
evento

92

In [299]:
df_processos[(df_processos['processo'] == processo)]

,arquivo,evento,processo,setor,numero_processo,ano_processo,ordem,data_resumo,resumo,sigilo,...,Inativa,DataPublicacao,IdInformacaoSubstituida,UsuarioInformacaoSubstituida,DataInformacaoSubstituida,IdTipoParecer,Observacao,IdProcesso,E_Relatorio_Inicial,E_Relatorio_Auditoria
140,PROC_LRC_700790_2012_0045.pdf,6,700790/2012,PROC_LRC,700790,2012,45,2017-11-08 14:02:38.847,............,None,...,None,2017-11-16 09:33:48.550,NaN,None,NaT,None,None,304368.0,None,None
208,GCTAR_700790_2012_0042.pdf,1,700790/2012,GCTAR,700790,2012,42,2017-07-31 11:58:51.860,Processo Digitalizado (Resolução 01/2012)- Vol...,0,...,None,2017-07-31 11:59:21.620,NaN,None,NaT,None,None,304368.0,None,None
235,PROC_700790_2012_0046.pdf,8,700790/2012,PROC,700790,2012,46,2017-11-16 14:29:28.230,Processo redistribuído ao Procurador,0,...,None,2017-11-16 14:29:28.240,NaN,None,NaT,None,None,304368.0,None,None
273,GCTAR_700790_2012_0044.pdf,3,700790/2012,GCTAR,700790,2012,44,2017-10-30 10:59:14.297,ENVIAR AO MP..................,0,...,None,2017-10-30 12:18:26.673,NaN,None,NaT,None,None,304368.0,None,None
296,GCTAR_700790_2012_0041.pdf,0,700790/2012,GCTAR,700790,2012,41,2017-07-31 11:58:51.670,Capa,None,...,True,2017-07-31 11:59:21.613,NaN,None,NaT,None,None,304368.0,None,None
306,DAE_MANDA_700790_2012_0033.pdf,48,700790/2012,DAE_MANDA,700790,2012,33,2017-02-14 09:34:40.277,Processo Transitou em Julgado,0,...,True,2024-02-20 21:13:08.040,NaN,None,NaT,None,None,304368.0,None,None
370,PROC_EXE_700790_2012_0047.pdf,10,700790/2012,PROC_EXE,700790,2012,47,2017-11-27 16:33:19.980,EMENTA: PROCESSUAL E ADMINISTRATIVO. PRESTAÇÃO...,None,...,None,2017-11-28 14:42:03.153,NaN,None,NaT,None,None,304368.0,None,None
385,GCTAR_700790_2012_0043.pdf,2,700790/2012,GCTAR,700790,2012,43,2017-07-31 11:58:54.460,Processo Digitalizado (Resolução 01/2012)- Vol...,0,...,None,2017-07-31 11:59:21.630,NaN,None,NaT,None,None,304368.0,None,None
445,DE_700790_2012_0049.pdf,0,700790/2012,DE,700790,2012,49,2019-01-25 13:48:19.780,Capa,None,...,True,2019-01-25 13:48:31.653,NaN,None,NaT,None,None,304368.0,None,None
498,DE_700790_2012_0048.pdf,14,700790/2012,DE,700790,2012,48,2019-01-25 13:48:19.667,Processo Distribuído ao Relator,0,...,None,2019-01-25 13:48:31.670,NaN,None,NaT,None,None,304368.0,None,None


In [300]:
df_processos[(df_processos.evento > evento) & \
                                (df_processos['processo'] == \
                                    processo)].sort_values(by='evento')

,arquivo,evento,processo,setor,numero_processo,ano_processo,ordem,data_resumo,resumo,sigilo,...,Inativa,DataPublicacao,IdInformacaoSubstituida,UsuarioInformacaoSubstituida,DataInformacaoSubstituida,IdTipoParecer,Observacao,IdProcesso,E_Relatorio_Inicial,E_Relatorio_Auditoria
5315,GCGEO_700790_2012_0096.pdf,95,700790/2012,GCGEO,700790,2012,96,2025-04-27 09:40:38.250,None,0,...,None,2025-04-30 11:16:43.197,NaN,None,NaT,None,None,NaN,None,None


In [243]:
evento

21